In [93]:
import pandas as pd
import pydicom
import pydicom as dicom
from PIL import Image
import numpy as np
import os
import cv2
import shutil
# from __future__ import print_function
from tqdm import tqdm
from shutil import copyfile
from numpy import nan as Nan
import cv2
import re

import matplotlib.pyplot as plt

# 1. Copy data without DICOM viewer

In [94]:
def check_wether_patient_has_records(path_to_patient_folder, get_names_of_records=False):
    
    """
    Args:
        get_names_of_records (bool): wether return names of record files
        
    Returns:
        bool: Retuns value. True if patient folder contains the records and False otherwise. 
        or
        tuple (bool, list): returns bool and names of the record files for the patient.  
    """
    names_of_the_records = [x for x in os.listdir(path_to_patient_folder) if 'doc' in x or 'xlsx' in x]
    if get_names_of_records:
        return len(names_of_the_records) >=1, names_of_the_records
    else:
        return len(names_of_the_records) >=1

def check_wether_patient_has_image_data(path_to_patient_folder):
    """
    Returns:
        bool : Returns True if patient folder contatin image data and False otherwise
    """
    names_of_the_records = [x for x in os.listdir(path_to_patient_folder) if 'DICOMOBJ' in x]
    return len(names_of_the_records) >= 1

def get_structure_of_the_dataset(path_to_dataset):
    """
    
    Returns:
        dict :  keys - patient names(str): values (list of strings) - paths to the images and records
                                                    
    """
    patients_database = {}
    
    reports_folders = [x for x in os.listdir(path_to_dataset) if not any(i in x for i in
                            ['System', 'BIN', '$.BIN', 'Autorun.inf','Seagate', 'SeagateExpansion.ico', 
                             'Start_Here_Mac.app', 'Start_Here_Win.exe', 'Warranty.pdf'])]
                       #'System' not in x and 'BIN' not in x]
    for report_folder in tqdm(reports_folders):
        patients_per_folder = os.listdir(os.path.join(path_to_dataset, report_folder))
        
        for patient in patients_per_folder:
            
            files_in_patient_folder = os.listdir(os.path.join(path_to_dataset, report_folder, patient))
            
            if check_wether_patient_has_image_data(os.path.join(path_to_dataset, report_folder, patient)):
                patient_images = os.listdir(os.path.join(path_to_dataset, report_folder, patient, 'DICOMOBJ'))
                patient_images_paths = [os.path.join(path_to_dataset, report_folder, patient, 'DICOMOBJ', x) 
                                 for x in patient_images]
            else:
                patient_images = []
                patient_images_paths = []
            _, patient_records = check_wether_patient_has_records(
                                      os.path.join(path_to_dataset, report_folder, patient), 
                                      get_names_of_records=True)
            patient_records_paths = [os.path.join(path_to_dataset, report_folder, patient, x) for x in patient_records]
            patients_database[patient] = []
            patients_database[patient] += patient_records_paths
            patients_database[patient] += patient_images_paths
    
    return patients_database

def copy_dataset(patients_database, path_to_copy):
    """
    Copy only image data and records without DICOM viewer program
    Args:
        patients_database (dict): dictionary with patients and corresponding 
                                  images and records
        path_to_copy (str): destination folder, where all dataset will
                            be located
        
    Returns:
        None
    """
    # Create folder to the dataset
    if not os.path.exists(path_to_copy):
        os.mkdir(path_to_copy)
    
    for patient in tqdm(patients_database):
        # Check wether patient's folder contains images
        if len(patients_database[patient]) <=2:
            continue
        # Check wether patient contains the records
        path_to_the_patient = patients_database[patient][0]

        path_to_the_patient = '\\'.join(path_to_the_patient.split('\\')[:4])
        if not check_wether_patient_has_records(path_to_the_patient):
            continue

        group_folder_name = patients_database[patient][0].split('\\')[0][2:]
        group_folder_name = '_'.join([x.lower() for x in group_folder_name.split()])
        
#         patient_folder_name = patients_database[patient][0].split('\\')[1]
        patient_folder_name = patient #'_'.join([x for x in patient_folder_name.split()])

        # Create directories
        if not os.path.exists(os.path.join(path_to_copy, group_folder_name)):
            os.mkdir(os.path.join(path_to_copy, group_folder_name))
        if not os.path.exists(os.path.join(path_to_copy, group_folder_name, patient_folder_name)):
            os.mkdir(os.path.join(path_to_copy, group_folder_name, patient_folder_name))
        
        # Copy Records
        shutil.copy(patients_database[patient][0], os.path.join(
                path_to_copy, group_folder_name, patient_folder_name, patients_database[patient][0].split('\\')[-1]))
        
        # Create folder patients's for images
        if not os.path.exists(os.path.join( path_to_copy, group_folder_name, patient_folder_name, 'images')):
            os.mkdir(os.path.join( path_to_copy, group_folder_name, patient_folder_name, 'images'))
            

        # Copy images
        for i in range(1, len(patients_database[patient])):
            shutil.copy(patients_database[patient][i], os.path.join(
                path_to_copy, patient_folder_name, 'images', patients_database[patient][i].split('\\')[-1]))



In [95]:
PATH_TO_THE_WHOLE_DATASET = 'D:\dataset'

In [ ]:
patients_database = get_structure_of_the_dataset(PATH_TO_THE_WHOLE_DATASET)
copy_dataset(patients_database, 'D:\data')


# 2. Take only MPRs

'AW electronic film', 'Scout', 'Correction Halted 80% - Series 304 - adjust vessel tracking'

'73','81', '71'

'45%','42%','80%',

'CTCA HD STD', 'CTCA','CTCA 73%','CTCA SMARTPHASE','CTCA 79%'

'CALCIUM SCORE', 'CALCIUM SCORE C2'

'SS-Freeze 39% - Original Series 306', 'Auto State - series 405 - 256 images','Auto State - series 301 - 256 images','Auto State - series 3 - 440 images', ,'Auto State - series 3 - 464 images', 'SS-Freeze 80% - Original Series 305','SS-Freeze 75% - Original Series 304','SS-Freeze 42% - Original Series 304','SS-Freeze 77% - Original Series 304','SS-Freeze 75% - Original Series 305','SS-Freeze 75% - Original Series 307','Auto State - series 405 - 256 images','SS-Freeze 39% - Original Series 306','SS-Freeze 39% - Original Series 306','SS-Freeze 77% - Original Series 305','SS-Freeze 81% - Original Series 305'

 '3D Saved State -  FINAL 76%'
 
 'SVG-PDA graft'
 
 
 'CALCIUM SCORE', 'CTCA', 'LAD *', 'LAD-D1 *', 'LAD-D2 *', 'OM *', 'PDA *', 'PLB *', 'PLB2 *', 'AW electronic film', 'Scout', 'LCX *', 'LCX-OM *', 'LAD-D3 *', 'LCX-OM2 *', 'PDA2 *', '73', 'RIB *', 'AM *', 'Correction Halted 80% - Series 304 - adjust vessel tracking', 'RCA *', 'RCA-AM *', 'PLB*', 'CTCA HD STD', '45%', 'RCA -PDA*', 'RCA -PLB*', 'RCA -AM*', 'OM2 *', 'RCA-PDA *', 'SS-Freeze 75% - Original Series 304', 'AM*', 'RIB-Branch of RIB *', 'LCX-OM1 *', 'CT ARTERIES', 'CTCA 73%', 'PLB1 *', '42%', 'SS-Freeze 42% - Original Series 304', 'SS-Freeze 77% - Original Series 304', 'LCX TO PDA *', 'LCX TO PDA2 *', 'LCX TO PLB *', 'AM1 *', 'AM2 *', '80%', 'LAD - D4 *', 'SS-Freeze 75% - Original Series 305', 'SS-Freeze 81% - Original Series 305', 'LCX-OM*', 'LCX-OM3 *', 'CALCIUM SCORE C2', '81', 'LCX - OM *', 'LCX - OM2 *', 'SS-Freeze 77% - Original Series 305', 'LAD', 'LAD-D1', 'LAD-D2', 'OM', 'OM2', 'RCA', 'PDA', 'SS-Freeze 75% - Original Series 307', 'RCA-PDA', 'RCA-PLB', 'OM1', 'OM3', 'RIB', 'CTCA SMARTPHASE', 'LCX', 'LCX-OM', 'PLB', 'LAD*', 'LAD-D1*', 'LCX TO PDA*', '71', 'OM3 *', 'PDA1 *', 'Branch 2 of PLB *', 'Branch of PLB *', 'LCX-OM2*', 'LCX-OM3*', 'RCA*', 'RCA-AM*', 'RCA-PDA*', 'RCA-PLB*', 'LAD-D2*', 'LCX*', 'LAD - D1', 'LCX-OM2', 'SS-Freeze 39% - Original Series 306', 'RCA-AM', 'RCA-PDA2', 'RCA-PLB2', 'RCA-PDA2*', 'Auto State - series 405 - 256 images', 'LCX-PLB', 'LCX-PDA', 'LAD-D3', 'LCX-OM1', 'LCX-OM3', 'LCX-PLB1', 'PDA2', '33', 'PLB2', 'PDA*', 'Auto State - series 3 - 256 images', 'CTCA 70%', 'Auto State - series 306 - 256 images', 'Auto State - series 301 - 256 images', '3D Saved State -  FINAL 76%', 'Auto State - series 304 - 256 images', 'LIMA-LAD', 'Aorta-D1', 'Aorta-D2', 'LAD original', 'LAD-D1 original', 'LCX original', 'LCX-OM original', 'PDA original', 'PDA2 original', 'PLB original', 'Aorta-LCX', 'Auto State - series 3 - 440 images', 'CTCA 79%', 'Auto State - series 305 - 256 images', 'Auto State - series 407 - 256 images', 'SS-Freeze 38% - Original Series 307', 'PLB1', 'LAD-D2 original', 'LAD-D3 original', 'OM original', 'RCA original', 'RCA-PDA original', 'RCA-PLB1 original', 'RCA-PLB2 original', 'LIMA-LAD graft', 'Aorta-LCX graft', 'Aorta-PDA graft', 'Auto State - series 3 - 464 images', 'Auto State - series 301 - 224 images', 'SS-Freeze 80% - Original Series 305', 'LAD-Branch of D1', 'LCX-Branch of OM', '59', 'Auto State - series 405 - 224 images', 'LCX-PDA2', 'RCA-AM1', 'RCA-AM2', 'OM4', 'LCX-PLB2', 'Auto State - series 304 - 224 images', 'LAD-D4', 'SS-Freeze 52% - Original Series 305', '70%', 'Auto State - series 304 - 376 images', 'SS-Freeze 78% - Original Series 305', 'RCA-PDA1', 'SS-Freeze 79% - Original Series 305', 'SS-Freeze 80% - Original Series 304', 'RCA-PLB1', 'Auto State - series 404 - 224 images', 'SS-Freeze 70% - Original Series 305', 'SS-Freeze 75% - Original Series 306', 'Auto State - series 406 - 224 images', 'Auto State - series 3 - 432 images', 'Auto State - series 403 - 256 images', 'SS-Freeze 75% - Original Series 303', 'LCX-OM4', 'RIB-Branch of RIB', 'Auto State - series 3 - 224 images', 'PDA1', 'OM-Branch of OM', 'SS-Freeze 74% - Original Series 305', 'LOCALISER', 'LAD Original', 'Auto State - series 3 - 456 images', 'LAD-D1 Original', 'Aorta-LAD graft', 'Aorta-D1 graft', 'SVG-PDA graft', 'SS-Freeze 45% - Original Series 306', 'Auto State - series 406 - 256 images', '80', 'SS-Freeze 45% - Original Series 310', 'Auto State - series 410 - 256 images', 'PDLB', 'SS-Freeze 33% - Original Series 305', 'SS-Freeze 50% - Original Series 305', '40', 'LCX-OM1*', 'SS-Freeze 51% - Original Series 308', 'SS-Freeze 72% - Original Series 309', 'RIB*', 'SS-Freeze 62% - Original Series 308', 'OM*', 'LAD-D3*', 'PDA2*', 'CALCIUM SCORE/70', 'SS-Freeze 70% - Original Series 306', 'LCX-PDA *', 'PLB2*', 'SS-Freeze 45% - Original Series 305', 'SS-Freeze 72% - Original Series 305', 'D1*', 'LCX-OM4 *', 'LCX-PLB *', 'RCA-PLB *', 'RCA-PDA1 *', 'RCA-PDA2 *', 'SS-Freeze 77% - Original Series 306', 'PLB3*', 'OM-Branch of OM *', '85', 'RCA-AM1 *', 'RCA-AM2 *', 'SS-Freeze 45% - Original Series 304', '43', 'LAD-Branch of D2 *', 'SS-Freeze 80% - Original Series 306', '75%', 'LAD TO PDA *', 'PDA TO LAD *', 'PDA3 *', 'RCA-PLB2 *', '28', 'AXIAL LUNG 2.5MM', '43%'
 



 
 
 'LCX-OM*', 'RCA-AM*', 'RCA-PDA*', 'RCA-PLB*', 'AW electronic film', 'Scout', 'CALCIUM SCORE', 'CTCA SMARTPHASE', 'LAD*', 'LAD-D1*', 'LAD-D2*', 'LCX*', 'LAD', 'LAD - D1', 'LCX', 'LCX-OM', 'LCX-OM2', 'RCA-PDA', 'RCA-PLB', 'LAD-D1', 'RIB', 'SS-Freeze 39% - Original Series 306', 'RCA-AM', 'LAD-D2', 'RCA-PDA2', 'RCA-PLB2', 'SS-Freeze 75% - Original Series 305', 'RCA-PDA2*', 'CTCA', 'Auto State - series 405 - 256 images', 'LCX-PLB', 'RCA', 'OM', 'LCX-PDA', 'LAD-D3', 'LCX-OM1', 'LCX-OM3', 'LCX-PLB1', 'PDA', 'PLB', 'PDA2', '33', 'PLB2', 'LCX-OM2*', 'LCX-OM3*', 'PDA*', 'PLB*', 'Auto State - series 3 - 256 images', 'CTCA 70%', 'Auto State - series 306 - 256 images', 'OM2', 'Auto State - series 301 - 256 images', '3D Saved State -  FINAL 76%'

In [118]:

def get_unique_mpr_names(path_to_dataset):
    """
    Prints all unique mpr section names.
    """
    folders = os.listdir(path_to_dataset)
    
    lol = ['78SSF', 'LOCALISER', 'Lt Septal Branch ', 'CT ARTERIES', 'Conus Artery *', 'AXIAL LUNG 2.5MM', 
           'LIMA-LAD graft', 'Aorta-LAD graft', 'RT Anomaly *', 'AXIAL MEDIASTINUM 2.5MM', 'Localiser', 'LIMA-LAD graft', 
           'SScoreSerSav', 'PatientReport', 'ECG Report', 'Dose Report', 'ScreenSave']
    thrash = 0
    lol2 = ['OM-Branch of OM *', 'LCX-Branch of OM *', 'RIB-Branch of RIB *', 'Branch of PLB *', 'Branch 2 of PLB *', 
            'OM-Branch of OM', 'LCX-Branch of OM', 'RIB-Branch of RIB', 'OM-Branch OM *', 'LAD-Branch of D1',
            'LCX-Branch of OM1 *', '81SSF', 'PLB3 *', 'LAD-Branch of D2 *', 'Branch of AM *']
    branch=0
    lol3 = ['Aorta-D1', 'Aorta-LCX', 'Aorta-D2', 'Aorta-OM graft*', 'Aorta-PDA graft *', 'Aorta-D1 graft *', 
            'Aorta-OM graft *','Aorta-LCX graft', 'Aorta-PDA graft', 'Aorta-D1 graft']
    aorta=0
    unique_modalities = []
    for patient_name in tqdm(os.listdir(os.path.join(path_to_dataset))):
        images = list(os.walk(os.path.join(path_to_dataset, patient_name)))[0][2]

        for image_name in images:
            dicom_obj = pydicom.dcmread(os.path.join(path_to_dataset,patient_name, image_name))
            
            if dicom_obj.SeriesDescription in lol:
                thrash+=1
            elif dicom_obj.SeriesDescription in lol2:
                branch+=1
            elif dicom_obj.SeriesDescription in lol3:
                aorta+=1
            
            if dicom_obj.SeriesDescription not in unique_modalities:
                unique_modalities.append(dicom_obj.SeriesDescription)
                
    return thrash, branch,aorta
#                     print(unique_modalities)     
#     print('       FINAL RESULTS:      ')
#     print(unique_modalities)
    
def copy_mpr_images_per_patient(path_to_dataset, path_to_save):
    """
    Creates in path_to_save folder for each patient, where all MPR DICOM files are located.
    """
    folders = os.listdir(path_to_dataset)
    raw_images = ['CTCA', 'CALCIUM SCORE', 'Scout', 'AW electronic film', '40', '81', '85','Freeze','Auto State','33','3D']
    for patient_name in tqdm(os.listdir(os.path.join(path_to_dataset))):
        images = list(os.walk(os.path.join(path_to_dataset, patient_name, 'images')))[0][2]
        if not os.path.exists(os.path.join(path_to_save,patient_name)):
            os.mkdir(os.path.join(path_to_save,patient_name))
        for image_name in images:
            dicom_obj = pydicom.dcmread(os.path.join(path_to_dataset,patient_name, 'images', image_name))
            if not any(sub_string in dicom_obj.SeriesDescription for sub_string in raw_images):
                copyfile(os.path.join(path_to_dataset,patient_name, 'images', image_name),
                         os.path.join(path_to_save,patient_name,image_name))
# copy_mpr_images_per_patient(PATH_TO_DATA, 'lol')

In [119]:
PATH_TO_DATA = '/home/petryshak/CoronaryArteryPlaqueIdentification/data/all_branches/ONLY_MPR'
PATH_TO_SAVE = r'D:\test\only_mprs'

In [120]:
thrash, branch,aorta = get_unique_mpr_names(PATH_TO_DATA)






  0%|          | 0/743 [00:00<?, ?it/s]




  0%|          | 1/743 [00:00<07:01,  1.76it/s]




  0%|          | 2/743 [00:01<07:23,  1.67it/s]




  0%|          | 3/743 [00:01<07:02,  1.75it/s]




  1%|          | 4/743 [00:02<07:39,  1.61it/s]




  1%|          | 5/743 [00:02<06:48,  1.81it/s]




  1%|          | 6/743 [00:03<07:06,  1.73it/s]




  1%|          | 7/743 [00:04<07:47,  1.57it/s]




  1%|          | 8/743 [00:05<08:55,  1.37it/s]




  1%|          | 9/743 [00:05<07:57,  1.54it/s]




  1%|▏         | 10/743 [00:06<07:15,  1.68it/s]




  1%|▏         | 11/743 [00:06<07:57,  1.53it/s]




  2%|▏         | 12/743 [00:07<07:21,  1.66it/s]




  2%|▏         | 13/743 [00:08<07:41,  1.58it/s]




  2%|▏         | 14/743 [00:08<07:09,  1.70it/s]




  2%|▏         | 15/743 [00:09<07:45,  1.56it/s]




  2%|▏         | 16/743 [00:10<07:54,  1.53it/s]




  2%|▏         | 17/743 [00:10<07:33,  1.60it/s]




  2%|▏         | 18/743 [00:11<07:22,  1.64it/s]




  3%|▎

 16%|█▌        | 120/743 [01:23<06:46,  1.53it/s]




 16%|█▋        | 121/743 [01:23<07:14,  1.43it/s]




 16%|█▋        | 122/743 [01:24<07:08,  1.45it/s]




 17%|█▋        | 123/743 [01:25<07:39,  1.35it/s]




 17%|█▋        | 124/743 [01:26<07:30,  1.37it/s]




 17%|█▋        | 125/743 [01:26<06:59,  1.47it/s]




 17%|█▋        | 126/743 [01:27<06:45,  1.52it/s]




 17%|█▋        | 127/743 [01:28<07:32,  1.36it/s]




 17%|█▋        | 128/743 [01:28<06:58,  1.47it/s]




 17%|█▋        | 129/743 [01:29<05:52,  1.74it/s]




 17%|█▋        | 130/743 [01:30<06:57,  1.47it/s]




 18%|█▊        | 131/743 [01:30<06:44,  1.51it/s]




 18%|█▊        | 132/743 [01:31<07:08,  1.43it/s]




 18%|█▊        | 133/743 [01:32<07:06,  1.43it/s]




 18%|█▊        | 134/743 [01:32<06:36,  1.54it/s]




 18%|█▊        | 135/743 [01:33<06:27,  1.57it/s]




 18%|█▊        | 136/743 [01:34<06:50,  1.48it/s]




 18%|█▊        | 137/743 [01:34<06:41,  1.51it/s]




 19%|█▊        | 138/743 [01

 32%|███▏      | 238/743 [02:45<05:40,  1.48it/s]




 32%|███▏      | 239/743 [02:45<05:16,  1.59it/s]




 32%|███▏      | 240/743 [02:46<06:20,  1.32it/s]




 32%|███▏      | 241/743 [02:47<06:53,  1.21it/s]




 33%|███▎      | 242/743 [02:48<06:12,  1.34it/s]




 33%|███▎      | 243/743 [02:48<05:48,  1.43it/s]




 33%|███▎      | 244/743 [02:49<05:37,  1.48it/s]




 33%|███▎      | 245/743 [02:50<05:26,  1.53it/s]




 33%|███▎      | 246/743 [02:50<05:13,  1.59it/s]




 33%|███▎      | 247/743 [02:51<05:11,  1.59it/s]




 33%|███▎      | 248/743 [02:52<05:17,  1.56it/s]




 34%|███▎      | 249/743 [02:52<05:53,  1.40it/s]




 34%|███▎      | 250/743 [02:53<06:02,  1.36it/s]




 34%|███▍      | 251/743 [02:54<05:38,  1.45it/s]




 34%|███▍      | 252/743 [02:54<05:26,  1.50it/s]




 34%|███▍      | 253/743 [02:55<06:05,  1.34it/s]




 34%|███▍      | 254/743 [02:56<05:57,  1.37it/s]




 34%|███▍      | 255/743 [02:57<05:24,  1.50it/s]




 34%|███▍      | 256/743 [02

 48%|████▊     | 356/743 [04:08<04:19,  1.49it/s]




 48%|████▊     | 357/743 [04:08<04:07,  1.56it/s]




 48%|████▊     | 358/743 [04:09<04:09,  1.54it/s]




 48%|████▊     | 359/743 [04:10<04:25,  1.45it/s]




 48%|████▊     | 360/743 [04:11<04:42,  1.35it/s]




 49%|████▊     | 361/743 [04:11<04:11,  1.52it/s]




 49%|████▊     | 362/743 [04:12<04:27,  1.42it/s]




 49%|████▉     | 363/743 [04:13<04:29,  1.41it/s]




 49%|████▉     | 364/743 [04:13<03:59,  1.58it/s]




 49%|████▉     | 365/743 [04:14<03:52,  1.63it/s]




 49%|████▉     | 366/743 [04:14<03:46,  1.67it/s]




 49%|████▉     | 367/743 [04:15<04:05,  1.53it/s]




 50%|████▉     | 368/743 [04:16<04:00,  1.56it/s]




 50%|████▉     | 369/743 [04:16<04:05,  1.53it/s]




 50%|████▉     | 370/743 [04:17<03:54,  1.59it/s]




 50%|████▉     | 371/743 [04:18<04:26,  1.40it/s]




 50%|█████     | 372/743 [04:19<05:25,  1.14it/s]




 50%|█████     | 373/743 [04:20<05:01,  1.23it/s]




 50%|█████     | 374/743 [04

 64%|██████▍   | 474/743 [05:31<03:40,  1.22it/s]




 64%|██████▍   | 475/743 [05:31<03:08,  1.42it/s]




 64%|██████▍   | 476/743 [05:32<03:11,  1.40it/s]




 64%|██████▍   | 477/743 [05:33<03:47,  1.17it/s]




 64%|██████▍   | 478/743 [05:34<03:34,  1.23it/s]




 64%|██████▍   | 479/743 [05:34<03:22,  1.31it/s]




 65%|██████▍   | 480/743 [05:35<03:09,  1.39it/s]




 65%|██████▍   | 481/743 [05:36<03:03,  1.43it/s]




 65%|██████▍   | 482/743 [05:36<03:02,  1.43it/s]




 65%|██████▌   | 483/743 [05:37<03:26,  1.26it/s]




 65%|██████▌   | 484/743 [05:39<03:55,  1.10it/s]




 65%|██████▌   | 485/743 [05:40<04:06,  1.05it/s]




 65%|██████▌   | 486/743 [05:40<03:53,  1.10it/s]




 66%|██████▌   | 487/743 [05:41<03:45,  1.14it/s]




 66%|██████▌   | 488/743 [05:42<03:42,  1.15it/s]




 66%|██████▌   | 489/743 [05:43<03:35,  1.18it/s]




 66%|██████▌   | 490/743 [05:44<03:32,  1.19it/s]




 66%|██████▌   | 491/743 [05:45<03:36,  1.16it/s]




 66%|██████▌   | 492/743 [05

 80%|███████▉  | 592/743 [06:57<02:09,  1.16it/s]




 80%|███████▉  | 593/743 [06:57<02:02,  1.23it/s]




 80%|███████▉  | 594/743 [06:58<01:57,  1.27it/s]




 80%|████████  | 595/743 [06:59<02:12,  1.11it/s]




 80%|████████  | 596/743 [07:00<01:57,  1.25it/s]




 80%|████████  | 597/743 [07:01<01:54,  1.27it/s]




 80%|████████  | 598/743 [07:01<01:41,  1.43it/s]




 81%|████████  | 599/743 [07:02<01:38,  1.46it/s]




 81%|████████  | 600/743 [07:02<01:37,  1.47it/s]




 81%|████████  | 601/743 [07:03<01:33,  1.52it/s]




 81%|████████  | 602/743 [07:04<01:38,  1.43it/s]




 81%|████████  | 603/743 [07:04<01:34,  1.49it/s]




 81%|████████▏ | 604/743 [07:05<01:34,  1.48it/s]




 81%|████████▏ | 605/743 [07:06<01:37,  1.42it/s]




 82%|████████▏ | 606/743 [07:07<01:33,  1.46it/s]




 82%|████████▏ | 607/743 [07:07<01:24,  1.61it/s]




 82%|████████▏ | 608/743 [07:07<01:18,  1.71it/s]




 82%|████████▏ | 609/743 [07:08<01:23,  1.61it/s]




 82%|████████▏ | 610/743 [07

 96%|█████████▌| 710/743 [08:20<00:26,  1.24it/s]




 96%|█████████▌| 711/743 [08:20<00:23,  1.37it/s]




 96%|█████████▌| 712/743 [08:21<00:20,  1.51it/s]




 96%|█████████▌| 713/743 [08:22<00:23,  1.27it/s]




 96%|█████████▌| 714/743 [08:23<00:20,  1.41it/s]




 96%|█████████▌| 715/743 [08:23<00:18,  1.49it/s]




 96%|█████████▋| 716/743 [08:24<00:19,  1.36it/s]




 97%|█████████▋| 717/743 [08:25<00:21,  1.24it/s]




 97%|█████████▋| 718/743 [08:26<00:20,  1.24it/s]




 97%|█████████▋| 719/743 [08:26<00:18,  1.32it/s]




 97%|█████████▋| 720/743 [08:27<00:17,  1.34it/s]




 97%|█████████▋| 721/743 [08:28<00:16,  1.34it/s]




 97%|█████████▋| 722/743 [08:28<00:14,  1.47it/s]




 97%|█████████▋| 723/743 [08:29<00:13,  1.44it/s]




 97%|█████████▋| 724/743 [08:30<00:14,  1.34it/s]




 98%|█████████▊| 725/743 [08:31<00:12,  1.48it/s]




 98%|█████████▊| 726/743 [08:31<00:11,  1.43it/s]




 98%|█████████▊| 727/743 [08:32<00:11,  1.40it/s]




 98%|█████████▊| 728/743 [08

In [ ]:
thrash, branch, aorta

In [ ]:
copy_mpr_images_per_patient(PATH_TO_DATA, PATH_TO_SAVE)

In [ ]:
'RCA-PDA','RCA -PDA', 'RCA-PDA*', 'RCA-PDA *'
'RCA-PDA1', 'RCA-PDA2', 'RCA-PDA2 *','RCA-PDA2', 'RCA-PDA2*'

'RCA-PLB', 'RCA-PLB ', 'RCA-PLB', 'RCA -PLB*'
'RCA-PLB1 *','RCA-PLB1', 'RCA-PLB1 *',
'RCA-PLB2', 'RCA-PLB2 *',

'RCA-PDA*', 'RCA-PDA *',



'LCX  TO PDA *', 'LCX TO PLB', 'LCX TO PDA', 'LCX TO PDA *', 'LCX TO PDA2 *', 'LCX TO PDA*', 'LCX-OM4'
'LCX-PLB', 'LCX-PLB *', 'LCX-PLB1', 'LCX-PLB2',  'LCX-PLB2 *'
'LCX-PDA *', 'LCX-PDA', 'LCX-PDA2', 'LCX-PDA2 *', 

'PLB  *', 'PLB *', 'PLB original', 'PLB','PLB*'
'PLB1 *', 'PLB1*', 'PLB1',
'PLB2 *', 'PLB2*', 'PLB2', 
'PLB3*', 'PLB3',

'PDA ', 'PDA original', 'PDA', 'PDA', 'PDA  *'
'PDA1 *',  
'PDA2', 'PDA 2', 'PDA2 original', 'PDA2 *', 'PDA2 *', 

### All unique values
'LAD-D2',  'LAD-D1', 'LAD', 'LAD *', 'LAD-D1 *', 'LAD-D2 *', , 'LAD-D3 *', 'LIMA-LAD', 'LAD-D1 original', 'LAD original', 'LAD*', 'LAD-D1*', 'LAD-D3', 'LAD-D2*', 'LAD-D3*', 'LAD - D4 *', 'LAD-D4', 'LAD TO PDA *', '2LAD-D2', 'LAD -D1*', 'LAD -D2*', 'LAD-D3 original', 'LAD-D2 original',  'LAD -D1', 'LAD -D2', 'LAD-D1 Original', 'LAD Original', 'LAD-D4 *', 'D1*', 'LAD - D1'



'RCA', 'RCA-PDA', 'PDA *', 'PDA2 original', 'PDA original', 'PDA2 *','RCA-AM *', 'RCA *', 'RCA-AM1', 'PDA', 'PDA1 *','PDA*','PDA3 *', 'PDA2','RCA*', 'PDA2*',  'PDA1*', 'PDA1', 'PDA 2*','PDA-AM *', 'RCA -PDA*'

'RCA-AM',

'RCA-PLB', 'RCA-PLB *', 'RCA-PLB*', 'RCA -PLB*', 'RCA-PLB1', 'RCA-PLB1 *', 'RCA-PLB2', 'RCA-PLB2 *',

'RCA-PDA*', 'RCA-PDA *', 'RCA-PDA1', 'RCA-PLB1 *', 'RCA-PDA2', 'RCA-PDA2 *'

'RCA -AM*', 'RCA-AM*', 'RCA-AM2 *', 'RCA-AM1 *', 'PDA TO LAD *', 'PDA2  *', 

'RCA-PLB1 original', 'RCA-PLB2 original', 'RCA original', 'RCA-PDA original'



'LCX', 'LCX-OM', 'LCX-OM *', 'LCX *', 'LCX-OM1 *', 'OM *', 'LCX-OM3 *', 'LCX-OM2 *', 'LCX-OM original', 'LCX original', 'OM2 *', 'LCX  *', 'LCX-OM  *', 'OM', 'OM1 *', 'LCX -OM3 *', 'LCX-OM*', 'LCX*', 'LCX-OM2', 'OM2', 'LCX2 *', 'LCX - OM2 *', 'LCX - OM *', 'OM3', 'OM1', 'LCX-OM3*', 'LCX-OM3', 'LCX-OM1', 'LCX-OM2*', 'OM3 *', 'OM3*',  'OM2*', 'OM4 *','LCX -OM2 *', 'LCX -OM1 *', 'OM4','LCX-OM4 *', 'LCX-OM1*', 'OM*', 'OM original', 

'LCX  TO PDA *', 'LCX TO PLB', 'LCX TO PDA', 'LCX TO PDA *', 'LCX TO PDA2 *', 'LCX TO PDA*', 'LCX-OM4'

'LCX-PLB', 'LCX-PLB *', 'LCX-PLB1', 'LCX-PLB2',  'LCX-PLB2 *'

'LCX-PDA *', 'LCX-PDA', 'LCX-PDA2', 'LCX-PDA2 *', 



'PLB  *', 'PLB *', 'PLB2 *', 'PLB original', 'PLB2*', 'PLB1 *', 'PLB', 'PLB*', 'PLB3*', 'PLB2', 'PLB1', 'PLB3', 'PLB1*', 'RCA-PDA2*'

'RIB *', 'RIB1 *', 'RIB2 *','RIB', 'RIB*','RIB2'

'AM *', 'AM1 *', 'AM2 *', 'RCA-AM2', 'AM*', 'AM1', 'AM2', 'LB *',  'AM'


### 500 images
'Aorta-D1', 'Aorta-LCX', 'Aorta-D2',  'Aorta-OM graft*', 'Aorta-PDA graft *', 'Aorta-D1 graft *', 'Aorta-OM graft *','Aorta-LCX graft', 'Aorta-PDA graft', 'Aorta-D1 graft'

-----------------------------------------------------------------------------------------------------------------------------

### 4000 images

'78SSF', 'LOCALISER', 'Lt Septal Branch *', 'CT ARTERIES', 'Conus Artery *', 'AXIAL LUNG 2.5MM', 'LIMA-LAD graft', 'Aorta-LAD graft', 'RT Anomaly *', 'AXIAL MEDIASTINUM 2.5MM', 'Localiser', 'LIMA-LAD graft*', 'SScoreSerSav',  'PatientReport', 'ECG Report', 'Dose Report', 'ScreenSave', 

------------------------------------------------------------------------------------------------------------------------------

### 1400 images
'OM-Branch of OM *', 'LCX-Branch of OM *', 'RIB-Branch of RIB *', 'Branch of PLB *', 'Branch 2 of PLB *', 'OM-Branch of OM',  'LCX-Branch of OM', 'RIB-Branch of RIB', 'OM-Branch OM *', 'LAD-Branch of D1', 'LCX-Branch of OM1 *', '81SSF', 'PLB3 *', 'LAD-Branch of D2 *', 'Branch of AM *', 'LAD-Branch of D1 *'

------------------------------------------------------------------------------------------------------------------------------
'PDLB'

, 'PDA  *'



# 3. Extract PNG images from the MPRs

In [ ]:
def map_mpr_name_to_record_name(mpr_name):
    main_branches_dict = {
        'LAD': ['LAD', 'LAD ', 'LAD Original', 'LAD original'],
        'D-1':['LAD-D1 original', 'LAD-D1 Original', 'LAD-D1', 'LAD-D1 *', 'LAD -D1', 'LAD -D1', 'LAD - D1', 'D1'],
        'D-2':['LAD-D2', 'LAD-D2 *', 'LAD-D2', '2LAD-D2', 'LAD -D2', 'LAD-D2 original', 'LAD -D2'],
        'D-3': ['LAD-D3', 'LAD-D3 *', 'LAD-D3', 'LAD-D3 original', ],
        'D-4': [ 'LAD - D4 *', 'LAD-D4', 'LAD-D4 *'],
        'RCA': ['RCA', 'RCA *', 'RCA*', 'RCA original'],
        'OM':['OM*', 'LCX-OM  *', 'OM *', 'OM', 'LCX-OM*', 'LCX - OM *', 'LCX-OM original', 'LCX-OM *', 'LCX-OM', 'OM original'],
        'OM-1': ['LCX-OM1 *', 'OM1 *', 'OM1', 'LCX-OM1', 'LCX -OM1 *', 'LCX-OM1*'],
        'OM-2': ['LCX-OM2 *', 'OM2 *', 'LCX-OM2', 'LCX - OM2 *', 'LCX -OM2 *', 'OM2*', 'LCX-OM2*'],
        'OM-3': ['LCX-OM3 *', 'LCX -OM3 *', 'OM3',  'LCX-OM3*', 'LCX-OM3', 'OM3 *', 'OM3*'],
        'OM-4': ['OM4 *', 'OM4', 'LCX-OM4 *'],
        'LCX': ['LCX', 'LCX *', 'LCX original', 'LCX  *', 'LCX*']
    }
    
    for key in main_branches_dict:
        if mpr_name in main_branches_dict[key]:
            return key

def split_mpr_name(mpr_name):
    return \
        "".join(mpr_name.split()).replace('*', '').replace('original', '') \
        .replace('LIMA-', '').replace('Branchof','').replace('TOPDA', '').replace('PDATO', '')

def get_patient_dictionary(path_to_patient_folder):
    """
    
    Returns dict of different types of images in the folder of patient. 
    
    Returns:
        dict: key - type of images; value - list of DICOM files, which sorted in the ascending order with restepct to the
                    depth of the image slice.
    """
    patient_dict = {}
    
    dicom_file_names = os.listdir(path_to_patient_folder)
    
    for i in range(len(dicom_file_names)):
        cur_dicom_obj = dicom.dcmread(os.path.join(path_to_patient_folder, dicom_file_names[i]))
        
        if cur_dicom_obj.SeriesDescription not in patient_dict.keys():
            patient_dict[cur_dicom_obj.SeriesDescription] = []
        patient_dict[cur_dicom_obj.SeriesDescription].append(cur_dicom_obj)
        
    # sort each type of images with respect to their depth in ascending order
    for i in patient_dict:
        patient_dict[i].sort(key=lambda x: x.InstanceNumber)
    
    return patient_dict

def get_pixels_hu(list_of_imgs):
    """
    Convert stack of the images into Houndsfeld units
    """
    image = np.stack([s.pixel_array for s in list_of_imgs])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 1
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = list_of_imgs[0].RescaleIntercept
    slope = list_of_imgs[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [ ]:
# path_to_data = r'D:\coronaryProject\dataset\binary_classification_MPR\images'
# path_to_new_data = r'E:\ONLY_LAD\\'
path_to_data = r'/home/petryshak/CoronaryArteryPlaqueIdentification/data/all_branches/ONLY_MPR'
path_to_new_data = r'/home/petryshak/CoronaryArteryPlaqueIdentification/data/all_branches/multibranch_dataset'
list_of_patients = os.listdir(path_to_data)
# patient_dictionary = get_patient_dictionary(path_to_data + '\\'+ list_of_patients[0])

In [ ]:
for i in tqdm(range(len(list_of_patients))):
    patient_dictionary = get_patient_dictionary(path_to_data + '/'+ list_of_patients[i])
    splited_mpr_names_filtered = [map_mpr_name_to_record_name(x) for x in patient_dictionary.keys()]
#     splited_mpr_names = [split_mpr_name(x) for x in patient_dictionary.keys()]
#     splited_mpr_names_filtered = [split_mpr_name(x).replace('LAD-', '').replace('D','D-').replace('AD-', 'AD') for x in patient_dictionary.keys() 
#                                   if 'LAD' in split_mpr_name(x)]
    dict_keys = list(patient_dictionary.keys())
    
    # change keys in the dict to the corresponding labels in the reports
    for k in range(len(dict_keys)):
#         patient_dictionary[split_mpr_name(key_element).replace('LAD-', '').replace('D','D-').replace('AD-', 'AD')] = \
#             patient_dictionary[key_element]
        if splited_mpr_names_filtered[k]:
            if dict_keys[k] in splited_mpr_names_filtered:
                pass
            else:
                patient_dictionary[splited_mpr_names_filtered[k]] = patient_dictionary[dict_keys[k]]
                del patient_dictionary[dict_keys[k]]
        else:
            del patient_dictionary[dict_keys[k]]

    if not os.path.exists(os.path.join(path_to_new_data, list_of_patients[i])):
        os.mkdir(os.path.join(path_to_new_data, list_of_patients[i]))

    for key in patient_dictionary.keys():
        
        for dicom_file in patient_dictionary[key]:
            if not os.path.exists(os.path.join(path_to_new_data, list_of_patients[i])):
                os.mkdir(os.path.join(path_to_new_data, list_of_patients[i]))
            
            if not os.path.exists(os.path.join(path_to_new_data, list_of_patients[i], key)):
                os.mkdir(os.path.join(path_to_new_data, list_of_patients[i], key))
# #             dicom_file.save_as(os.path.join(path_to_new_data, 
# #                                             list_of_patients[i], 
# # #                                             key,
# # #                                             list_of_patients[i]+'_'+str(dicom_file.InstanceNumber)
# # #                                            )
# # #                               )
            cv2.imwrite(os.path.join(path_to_new_data, 
                                            list_of_patients[i], 
                                            key,
                                            list_of_patients[i]+'_'+str(dicom_file.InstanceNumber)+'.png'
                                           ),
                        cv2.normalize(dicom_file.pixel_array, None, alpha = 0, 
                                      beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
                       )

    

In [ ]:
def copy_mpr_records(path_to_dataset, path_to_save):
    """
    Copy all records from the dataset to path_to_save folder.
    """
    if not os.path.exists(path_to_save):
        os.mkdir(path_to_save)
    folders = [x for x in os.listdir(path_to_dataset) if 'WITH RECONS ' in x]

    for patient_name in os.listdir(os.path.join(path_to_dataset)):
        files = os.listdir(os.path.join(path_to_dataset, patient_name))
        files = [x for x in files if ('xlsx' in x) or ('doc' in x)]
        files = files[0] if len(files)>0 else None
        if files:
            copyfile(os.path.join(path_to_dataset, patient_name, files), os.path.join(path_to_save, files))


In [ ]:
PATH_TO_DATA = r'd:\test\without_viewer'
PATH_TO_SAVE = r'd:\test\records'

In [ ]:
copy_mpr_records(PATH_TO_DATA, PATH_TO_SAVE)

# 5. Merge Reports

In [ ]:
def strip_mpr_lad_name(mpr_name):
    """
    Strip MPR name of the LAD artery. We do this step because the name in the doctor's report 
    is not equal to the name in the MPR. 
    
    Returns:
        - str: striped string
    """
    return "".join(mpr_name.split()).replace('*', '').replace('original', '')

def read_and_strip_record(path_to_record):
    '''
    Read record file and remove empty rows and rows with all NaNs.
    
    Returns:
        - Pandas DataFrame: 
    '''
    excel_file = pd.read_excel(path_to_record,index_col=None, header=None)
    excel_file.dropna(how='all')
    excel_file.rename(columns={0: 'a', 1: 'b'}, inplace=True)
    excel_file = excel_file.fillna('  ')
    excel_file = excel_file.replace('', '  ', regex=True)
    excel_file = excel_file.drop(excel_file[excel_file['a'].str.isspace()].index)
    return excel_file

def get_lad_info_from_report(striped_record, artery_type):
    """
    Takes striped(without any empty lines and NaNs) and returns info only about the certain artery type. 
    
    Returns:
        - list: each element is the string with some info about certain artery type
    """
    lad_info = []
    wether_add = False
    lad_info.append(striped_record.iloc[0]['b'])
    for ind, row_value in striped_record.iterrows():
        
        if wether_add and row_value['a'].isupper():
            break
        if wether_add:
            lad_info.append(row_value['a'])
        
        if artery_type in row_value['a']:
            wether_add = True
    return lad_info

def get_level_of_stenosis_from_string(artery_info):
    """
    Returns:
        - list of str: each element is the string with percentage of stenosis. 
    """
    return [x.strip() for x in re.findall(r'.\d{1,3}.?\d{1,3}\%', artery_info)]

In [ ]:
path_to_records = r'/home/petryshak/CoronaryArteryPlaqueIdentification/data/reports'
list_of_files = os.listdir(path_to_records)

In [ ]:
extracted_lad_df = pd.DataFrame(columns=['PATIENT_ID','PROXIMAL', 'MID', 'DISTAL', 'D-1', 'D-2', 'D-3', 'D-4'])

for i in tqdm(range(len(list_of_files))):
    cur_file = read_and_strip_record(os.path.join(path_to_records, list_of_files[i]))
    cur_patient_lad_info = get_lad_info_from_report(cur_file, 'LEFT ANTERIOR')
    
    new_row = pd.Series(#[Nan,Nan,Nan,Nan,Nan,Nan,Nan,Nan],
                        ['-','-','-','-','-','-','-','-'],
                        index=extracted_lad_df.columns)
    new_row['PATIENT_ID'] = cur_patient_lad_info[0]
    cur_patient_lad_info.pop(0)
    list_of_lda_branches = list(extracted_lad_df.columns)
    
    for line_info in cur_patient_lad_info:
        
        artery_area_name = [x for x in list_of_lda_branches 
                            if x in line_info or x.lower() in line_info or x.title() in line_info]
        if len(artery_area_name) >=1:
            artery_area_name = artery_area_name[0]
        else:
            continue
        stenosis_score = get_level_of_stenosis_from_string(line_info)
        stenosis_score =  stenosis_score[0] if stenosis_score else 'NORMAL'
        new_row.loc[artery_area_name] = stenosis_score
    extracted_lad_df = extracted_lad_df.append(new_row, ignore_index=True)

In [ ]:
extracted_lad_df.head()

In [ ]:
extracted_lad_df.to_excel('lad_reports.xlsx', index=False)

# 6. Remove text from the images

In [ ]:
def remove_text(img):
    mask = cv2.threshold(img, 250, 255, cv2.THRESH_BINARY)[1][:,:,0]
    dilated_mask = cv2.dilate(mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)))
    dst = cv2.inpaint(img, dilated_mask, 5, cv2.INPAINT_NS)
    return dst


In [ ]:
PATH_TO_DATA = '/home/petryshak/CoronaryArteryPlaqueIdentification/data/all_branches/multibranch_dataset'
patients_names = os.listdir(PATH_TO_DATA)

for patient_name in tqdm(patients_names):
    images_types = os.listdir(os.path.join(PATH_TO_DATA, patient_name))
    
    for img_type in images_types:
        images_names = os.listdir(os.path.join(PATH_TO_DATA, patient_name, img_type))
        filtered_imgs = [x for x in images_names if '_text_deleted' in x]
        if len(filtered_imgs) > 0:
            continue
            
        for img_name in images_names:
            img_path = os.path.join(PATH_TO_DATA, patient_name, img_type, img_name)
            cur_img = cv2.imread(img_path)
            img_without_text = remove_text(cur_img)
            cv2.imwrite(img_path, img_without_text)
#             cv2.imwrite(os.path.join(PATH_TO_DATA, patient_name, img_type, img_name.split('.')[0] + '_text_deleted.png'), img_without_text)


# 7. Create binary dataset

In [96]:
def get_label_for_patient_lad(lad_segment, reports, patient_id):
    """
    
    
    Example:
        get_label_for_patient('LAD', reports, 'DDJ261Z' )

    """
    classes_to_positive = ['<25%', 'NORMAL', '-']

    if lad_segment == 'LAD':
        curr_section_label = reports.loc[reports['PATIENT_ID'] == patient_id][['MID', 'PROXIMAL', 'DISTAL']].iloc[0]
        stenosis_score = [x for x in curr_section_label if x not in classes_to_positive]
        label = 1 if len(stenosis_score) > 0 else 0
        return label, '___'.join(curr_section_label.values)
        
    else:
        curr_section_label =  reports.loc[reports['PATIENT_ID'] == patient_id][lad_segment].iloc[0]
        label = 0 if curr_section_label in classes_to_positive else 1    
        return label, curr_section_label

def get_label_for_patient(artery_segment, reports, patient_id):
    """
    
    
    Example:
        get_label_for_patient('LAD', reports, 'DDJ261Z' )

    """    
    classes_to_positive = ['<25%', 'NORMAL', '-']
    curr_section_label = reports.loc[reports['PATIENT_ID'] == patient_id][artery_segment].iloc[0]
    label = 0 if curr_section_label in classes_to_positive else 1    
    return label, curr_section_label


    
def get_labels(path_to_patient, reports):
    """
    Returns:
        - dict: key(str) - type of the artery, value(int) - label(0 or 1)
    """
    
    types_of_images = os.listdir(path_to_patient)
    types_of_images = [x for x in types_of_images if x in list(reports.columns)]
    patient_name = [x for x in path_to_patient.split('/') if len(x) > 0][-1]
    labels_dict = {}
    
    for i in range(len(types_of_images)):
        labels_dict[types_of_images[i]] = get_label_for_patient(types_of_images[i], reports, patient_name)
        
    return dict(filter(lambda elem: elem[1][1] != '-', labels_dict.items()))


def get_imgs_names_to_the_labels(path_to_patient, labels_dict):
    """
    Args:
        - dict: key - branch of artery(str), value - label(int) 0 or 1.   
    Returns:
        - dict: key - label(int) 0 or 1, value(list) list of images, belong to the labelT
    """
    img_labels = {}
    for key in labels_dict.keys():
        list_of_images = os.listdir(os.path.join(path_to_patient, key))
        
        if labels_dict[key] in img_labels:
            new_key = (key, labels_dict[key][1], labels_dict[key][0])
#             img_labels[labels_dict[key]]+= list_of_images
            img_labels[new_key] += list_of_images
        else:
            new_key = (key, labels_dict[key][1], labels_dict[key][0])
            img_labels[new_key] = list_of_images

#             img_labels[labels_dict[key]] = list_of_images
            
    return img_labels


In [24]:
get_labels('/home/petryshak/CoronaryArteryPlaqueIdentification/data/binary_classification_all_branches/train/imgs/CTCAZOF20051940',
           reports)

{'D-1': (0, 'NORMAL'),
 'D-2': (0, 'NORMAL'),
 'D-3': (0, 'NORMAL'),
 'OM-2': (0, 'NORMAL'),
 'RCA': (0, 'NORMAL')}

### A) rename name of the patient's folders

In [97]:
PATH_TO_MERGED_CSV = '/home/petryshak/CoronaryArteryPlaqueIdentification/notebooks/ExtractLabels/lad_rca_lcx.xlsx'

PATH_TO_RENAME = '/home/petryshak/CoronaryArteryPlaqueIdentification/data/all_branches/multibranch_dataset'


In [98]:
reports = pd.read_excel(PATH_TO_MERGED_CSV, index_col=0)
# file_names = os.listdir(PATH_TO_RENAME)
file_names_reports = list(reports['PATIENT_ID'])

In [ ]:

new_values = []
exceptions_which_should_be_added = \
    [
     '348 CTCAGRH27071943',
     '349 CTCANGM17081945',
     '350 CTCATRH10061944',
     '351 CTCAGRH27071943',
     '353 CTCANGM17081945',
     '371 CTCATRH10061944'
    ]

for i in range(len(file_names)):
    if file_names[i] in exceptions_which_should_be_added:
        continue
    
    splited_value = file_names[i].split(' ')
    if len(splited_value[0]) == 4 or file_names[i] in exceptions_which_should_be_added:
        new_values.append(file_names[i])
    else:
        os.rename(os.path.join(PATH_TO_RENAME, file_names[i]), os.path.join(PATH_TO_RENAME, splited_value[1]))
        new_values.append(splited_value[1])

### B) Rename all images per patient

In [ ]:
PATH_TO_DATA = '/home/petryshak/CoronaryArteryPlaqueIdentification/data/all_branches/multibranch_dataset'

patients_names = os.listdir(PATH_TO_DATA)

for patient_name in tqdm(patients_names):
    images_types = os.listdir(os.path.join(PATH_TO_DATA, patient_name))
    
    for img_type in images_types:
        images_names = os.listdir(os.path.join(PATH_TO_DATA, patient_name, img_type))
        for img_name in images_names:
            os.rename(os.path.join(PATH_TO_DATA, patient_name, img_type, img_name),
                      os.path.join(PATH_TO_DATA, patient_name, img_type, img_type+'_'+'_'.join(img_name.split(' '))))

### C) Train/Val/Split

In [ ]:
def move_dataset_part(dataset_part_name, patients_list, path_to_data , path_to_move):
    """
    Takes patient list and move to new subdataset.
    
    Args:
        - dataset_part_name(str): name of the new subpart of the dataset
        - patients_list: list of the patient, which sould be moved
        - path_to_data: dataset, from which we take the patient folders
        - path_to_move: path, where create new dataset
    Returns:
        - None
    """
    if not os.path.exists(os.path.join(path_to_move, dataset_part_name)):
        os.mkdir(os.path.join(path_to_move, dataset_part_name))
    
    for i in range(len(patients_list)):
        shutil.move(os.path.join(path_to_data, patients_list[i]), os.path.join(path_to_move, dataset_part_name, patients_list[i]))

349 CTCANGM17081945 - deleted because it empty

In [ ]:
test_part = [
    'CTCAZHX30011957',
    'CTCAYOG08091955',
    'CTCAYOA13121966',
    'CTCATHJ17011957',
    'CTCAPHD16081938',
    'CTCASIP30041975',
    'CTCASTW15121946',
    'CTCATKR01031953',
    'CTCAVAH09071948',
    'CTCAZDV13081958',
]

val_part = [
    '1006 CTCA1961',
    '1007 CTCA1959',
    '1009 CTCA1955',
    '1027 CTCA1965',
    'CTCAAGK05031979',
    'CTCAANM18021961',
    '1001 CTCA1947',
    '1002 CTCA1955',
    '1060 CTCA1959',
    '1038 CTCA1979',
    '1045 CTCA1950',  
    'CTCAWUK05041963',
    'CTCASTR17021954',
    'CTCASTS01111969',
    'CTCATUQ02091955',
]


In [ ]:
PATH_TO_DATA = '/home/petryshak/CoronaryArteryPlaqueIdentification/data/binary_classification_all_branches/train'
PATH_TO_MOVE = '/home/petryshak/CoronaryArteryPlaqueIdentification/data/binary_classification_all_branches'
DATASET_PART_NAME = 'test'
move_dataset_part(DATASET_PART_NAME, test_part, PATH_TO_DATA, PATH_TO_MOVE)

In [ ]:
PATH_TO_DATA = '/home/petryshak/CoronaryArteryPlaqueIdentification/data/binary_classification_all_branches/train'
PATH_TO_MOVE = '/home/petryshak/CoronaryArteryPlaqueIdentification/data/binary_classification_all_branches'
DATASET_PART_NAME = 'val'
move_dataset_part(DATASET_PART_NAME, val_part, PATH_TO_DATA, PATH_TO_MOVE)

### D) Create final dataset


In [ ]:
train_files = os.listdir('/home/petryshak/CoronaryArteryPlaqueIdentification/data/binary_classification_only_lad/train')
val_files = os.listdir('/home/petryshak/CoronaryArteryPlaqueIdentification/data/binary_classification_only_lad/val')
test_files = os.listdir('/home/petryshak/CoronaryArteryPlaqueIdentification/data/binary_classification_only_lad/test')


In [7]:
reports.tail()

,PATIENT_ID,REPORT_ID,LAD,D-1,D-2,D-3,RCA,OM,OM-1,OM-2,OM-3,LCX
739,CTCALAN04091968,CTCALAN04091968,NORMAL,-,-,-,NORMAL,-,NORMAL,-,-,NORMAL
740,1058 CTCA1956,1058 CTCA1956,50-70%,NORMAL,-,-,NORMAL,-,NORMAL,-,-,NORMAL
741,CTCABOJ29051945,CTCABOJ29051945,NORMAL,NORMAL,NORMAL,-,NORMAL,-,NORMAL,-,-,NORMAL
742,CTCAGRC24011965,CTCAGRC24011965,NORMAL,NORMAL,NORMAL,NORMAL,NORMAL,-,NORMAL,-,-,NORMAL
743,CTCAGAR01101953,CTCAGAR01101953,25%,NORMAL,-,-,NORMAL,-,NORMAL,NORMAL,NORMAL,NORMAL


In [107]:
PATH_TO_DATA = r'/home/petryshak/CoronaryArteryPlaqueIdentification/data/binary_classification_all_branches/train/imgs'
list_of_patients = os.listdir(PATH_TO_DATA)

In [113]:

labels_csv = pd.DataFrame(columns=['PATIENT_NAME', 'IMG_NAME', 'STENOSIS_SCORE', 'ARTERY_SECTION', 'MPR_VIEWPOINT_INDEX', 'LABEL'])
# labels_csv = pd.DataFrame(columns=['IMG_PATH', 'STENOSIS_SCORE', 'ARTERY_SECTION', 'MPR_VIEWPOINT_INDEX', 'LABEL'])

for i in tqdm(range(len(list_of_patients))):
#     if i == 0:
#         continue
    if list_of_patients[i] in fif: #'1006 CTCA1961':#fif:
        continue
    
    print(list_of_patients[i])
    labels = get_labels(os.path.join(PATH_TO_DATA, list_of_patients[i]), reports)
    dict_labels_images = get_imgs_names_to_the_labels(os.path.join(PATH_TO_DATA, list_of_patients[i]), labels)

    for key in dict_labels_images:
        each_ind = 0
        for j in range(len(dict_labels_images[key])):
#             if each_ind % 3 == 0:
            mpr_veiwpoint_index = dict_labels_images[key][j].strip('.png').split('_')[-1]
                                        
#             new_row = pd.Series(
#                                 [
#                                     os.path.join('imgs/', list_of_patients[i], key[0], dict_labels_images[key][j]),
#                                     key[1],
#                                     key[0],
#                                     mpr_veiwpoint_index,
#                                     key[2]
#                                 ],
#                                 index=labels_csv.columns)
            new_row = pd.Series(
                                [
                                    list_of_patients[i],
                                    dict_labels_images[key][j],
                                    key[1],
                                    key[0],
                                    mpr_veiwpoint_index,
                                    key[2]
                                ],
                                index=labels_csv.columns)
            labels_csv = labels_csv.append(new_row, ignore_index=True)
#             each_ind+=1            









  0%|          | 0/718 [00:00<?, ?it/s]

CTCATHR12061965







  0%|          | 1/718 [00:00<05:03,  2.37it/s]

1042 CTCA1963







  0%|          | 2/718 [00:00<04:26,  2.69it/s]

...






100%|█████████▉| 716/718 [11:18<00:03,  1.80s/it]

CTCABAS25051953







100%|█████████▉| 717/718 [11:21<00:02,  2.14s/it]

CTCANAS04021967







100%|██████████| 718/718 [11:24<00:00,  2.21s/it]

In [114]:
labels_csv.head()

,PATIENT_NAME,IMG_NAME,STENOSIS_SCORE,ARTERY_SECTION,MPR_VIEWPOINT_INDEX,LABEL
0,CTCATHR12061965,D-3_366_CTCATHR12061965_27.png,NORMAL,D-3,27,0
1,CTCATHR12061965,D-3_366_CTCATHR12061965_15.png,NORMAL,D-3,15,0
2,CTCATHR12061965,D-3_366_CTCATHR12061965_11.png,NORMAL,D-3,11,0
3,CTCATHR12061965,D-3_366_CTCATHR12061965_18.png,NORMAL,D-3,18,0
4,CTCATHR12061965,D-3_366_CTCATHR12061965_13.png,NORMAL,D-3,13,0


In [115]:
labels_csv.to_csv('train_labels.csv')#, index=False)

In [41]:
labels_csv['STENOSIS_SCORE'].shape[0] / 3

NameError: name 'labels_csv' is not defined

In [ ]:
pd.value_counts(labels_csv['STENOSIS_SCORE'])

In [ ]:
labels_csv.head()

In [58]:
fif.append('CTCAANM20091965')

In [108]:
fif = []
for i in range(len(list_of_patients)):
    if list_of_patients[i] not in list(reports['PATIENT_ID']):
        fif.append(list_of_patients[i])
        print(list_of_patients[i])

349 CTCANGM17081945
CTCASAA13011957
CTCABAM18061976
CTCASIN11111959
CTCAPUP1208191941
CTCADOP23111947
CTCAKOG14041971
CTCACOV10031966
CTCACHY14101949
CTCALAT23091942
1003 CTCA1952
371 CTCATRH10061944
1000 CTCANEJ23051970
353 CTCANGM17081945
348 CTCAGRH27071943
351 CTCAGRH27071943
CTCATRM02031956
CTCAROS28091767
CTCASTV03081936
CTCATAA17041965
350 CTCATRH10061944
CTCAESL01121969
CTCAIKE28031986
CTCAFEY20011971


In [112]:
test_set_names = os.listdir(r'D:\extended_test_set')

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\extended_test_set'